<a href="https://colab.research.google.com/github/lisaong/stackup-workshops/blob/text_similarity/text-similarity/text_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>